In [25]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "https://localhost:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## Hera - compiles workflow manifest

In [47]:
from hera.workflows import Workflow, DAG, script, Artifact, Parameter, models as m, TaskResult

@script(outputs=[Parameter(name="coin",value_from=m.ValueFrom(path="./coin_output.txt"))])
def set_coin(coin: str):
    # import random
    # res = "heads" if random.randint(0, 1) == 0 else "tails"
    with open('./coin_output.txt','w') as output:
        output.write(coin)
        
@script()
def show_coin(coin: str):
    print(f"it was {coin}")
    
with Workflow(generate_name="set-a-coin-", entrypoint="Coin-set",namespace="argo",arguments=[Parameter(name="coin",value="heads")]) as w:
    with DAG(name="Coin-set"):
        set_coin_task = set_coin(name="Set-a-coin",arguments=[w.get_parameter('coin').as_argument()])
        
        show_coin_task = show_coin(name="Show-a-coin",arguments=[set_coin_task.get_parameter('coin').as_argument()],depends='Set-a-coin') # accesses the stdout from the fc task and checks if its "heads"

In [48]:
import yaml

with open('hera-pipeline-2.yaml','w') as pipeline_file:
    yaml.dump(w.to_yaml(),pipeline_file)

# Hera - submit pipeline

In [49]:
w.create()

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Workflow(api_version=None, kind=None, metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2024, 4, 30, 23, 3, 26, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='set-a-coin-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2024, 4, 30, 23, 3, 26, tzinfo=datetime.timezone.utc)))], name='set-a-coin-c4djv', namespace='argo', owner_references=None, resource_version='46653', self_link=None, uid='4654934c-d58d-45dd-987e-db65ca3b5b67'), spec=WorkflowSpec(active_deadline_seconds=None, affinity=None, archive_logs=None, arguments=Arguments(artifacts=None, parameters=[Parameter(default=None, description=None, en